In [2]:
from sklearn.preprocessing import StandardScaler
from visualization_fct import *

import itertools
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import show  # output_file, save


from pomegranate import HiddenMarkovModel
from pomegranate import MultivariateGaussianDistribution
from pomegranate import State, Distribution

import matplotlib.pyplot as plt  # , mpld3

data = pd.read_csv("../asm_data/asm_data_for_ml.txt", sep='\t')
del data['MJD']
del data['error']
del data['errorA']
del data['errorB']
del data['errorC']
data['rateCA'] = data.rateC / data.rateA
data_thr = mask(data, 'orbit')  # rm too large values except for 'orbit'

Loading BokehJS ...

In [ ]:
import matplotlib.animation as animation

color_key = ["red", "blue", "yellow", "grey", "black", "purple", "pink",
             "brown", "green", "orange"]

fig, ax = plt.subplots()
ax.set_xlim(0, 5)
ax.set_ylim(0, 120)
ax.set_title('ASM data over time with human annotations')

XX = np.c_[data_thr.rateCA, data_thr.rate]

x = XX[:, 0]
y = XX[:, 1]

upper = np.array(y > np.maximum(20, 350 * (x - 0.28))).astype(bool)
lower = np.array(y < np.maximum(20, 55 * (x - 0.28))).astype(bool)

yy = np.ones(XX.shape[0]).astype(int)
yy[lower] = 0
yy[upper] = 2

line = ax.scatter(XX[:, 0], XX[:, 1])

# plot the manual decisions (lines):
t = np.linspace(0, 2, 1000)
y1 = np.max([20 * np.ones(t.shape[0]), 55 * (t - 0.28)], axis=0)
tt = t[:300]
y2 = np.max([20 * np.ones(tt.shape[0]), 350 * (tt - 0.28)], axis=0)
ax.plot(t, y1, color='black')
ax.plot(tt, y2, color='black')


def animate(i):
    line.set_offsets(XX[10*i:10*(i+1), :])
    #line.set_xdata(XX[10*i:10*(i+1), 0])  # update the data
    #line.set_ydata(XX[10*i:10*(i+1), 1])   # update the data
    #line.set_color(color_key[preds[1*i]])
    color = [color_key[yy[10*i + k]] for k in range(10)]
    line.set_color(color)
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(4000, 6000), #init_func=init,
                              interval=1, blit=True)
#plt.show()
ani.save('streaming_asm.mp4', writer = 'mencoder', fps=30, extra_args=['-vcodec', 'libx264'])